In [2]:
import pandas as pd
import numpy as np
three_margin_hopen_mer = pd.read_pickle('three_margin_hopen_mer.pk')
com_rev_mon_y = pd.read_pickle('com_rev_mon_y.pkl')
kiki_merge_new = pd.read_pickle('kiki_merge_new.pk')

In [3]:
com_rev_mon_y['date'] = pd.to_datetime(com_rev_mon_y['date'])

In [4]:
com_rev_mon_y.drop(['stock_status','market_status','up_or_down'],axis = 1,inplace = True)
kiki_merge_new.drop(['up_or_down'],axis = 1,inplace = True)
three_margin_hopen_mer.drop(['up_or_down'],axis = 1,inplace = True)

In [5]:
pop = pd.merge(com_rev_mon_y,kiki_merge_new,on=['stock','date'],how='left')
pop = pd.merge(pop,three_margin_hopen_mer,on=['stock','date'],how='left')

In [6]:
stock_status = pd.read_pickle('stock_status_new.pk')

In [7]:
stock_status.columns

Index(['stock', 'date', 'up_or_down', 'up_or_down_150'], dtype='object')

In [8]:
stock_status['date'] = pd.to_datetime(stock_status['date'])

In [9]:
stock_status = stock_status.drop(['up_or_down'],axis=1)

In [10]:
pop = pd.merge(pop,stock_status,on=['stock','date'],how='left')

In [11]:
list_150 = pd.read_excel("150_list.xlsx")
list_150['stock'] = list_150['stock'].astype(str)
pop = pd.merge(pop,list_150,on=['stock'],how='inner')

In [12]:
pop.drop(['abt_rev_29','stock_y','abt_rev_13','abt_rev_16'],axis = 1,inplace = True)

In [13]:
mapping = {'H':1,'L':0}
pop['abt_rev_18'] = pop['abt_rev_18'].map(mapping)
pop['abt_rev_19'] = pop['abt_rev_19'].map(mapping)
pop['abt_rev_20'] = pop['abt_rev_20'].map(mapping)
pop['abt_rev_96'] = pop['abt_rev_96'].map(mapping)

In [14]:
pop.dtypes[pop.dtypes == 'object']
# pop[['abt_rev_18','abt_rev_19','abt_rev_20','abt_rev_96']]
# pop.abt_rev_96.value_counts()
# pop.info()

stock    object
dtype: object

In [39]:
train = pop[pop['date'].between("2017-04-01","2018-03-31")]
val = pop[pop['date'].between("2018-04-01","2018-06-30")]
test = pop[pop['date'].between("2018-07-01","2018-09-30")]

In [40]:
train.up_or_down_150.value_counts()

0.0    4109
1.0    3535
Name: up_or_down_150, dtype: int64

In [41]:
X_train = train.drop(['date','stock','up_or_down_150'],axis = 1)
y_train = train['up_or_down_150']
X_val = val.drop(['date','stock','up_or_down_150'],axis = 1)
y_val = val['up_or_down_150']
X_test = test.drop(['date','stock','up_or_down_150'],axis = 1)
y_test = test['up_or_down_150']

In [42]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier(
#         num_leaves=50,
#         max_depth =8,
#           learning_rate=0.01,
#           n_estimators=1000
#         min_child_samples = 20,
#         colsample_tree = 0.6,
#         eval_set=[(X_val, y_val)]
        )
gbm.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
# from sklearn.metrics import classification_report
# X_tra_pred = gbm.predict(X_train)
# print(classification_report(y_train,X_tra_pred))
# X_val1_pred = gbm.predict(X_val1)
# print(classification_report(y_val1,X_val1_pred))
# X_val2_pred = gbm.predict(X_val2)
# print(classification_report(y_val2,X_val2_pred))
# X_val3_pred = gbm.predict(X_val3)
# print(classification_report(y_val3,X_val3_pred))

In [44]:
importances = pd.DataFrame(gbm.feature_importances_,index=X_train.columns,columns=['imporatance']).sort_values('imporatance',ascending=False)
print(importances[importances['imporatance'] > 0])
var_list = list(importances[importances['imporatance'] > 0].index)

                             imporatance
DL_BS_mean                            31
SSS_diff_ratio_min                    31
SSS_mt_ratio_min                      29
BSM_mt_ratio_min                      27
Trade_Value_max                       26
DL_Pro_BS_mean                        25
BSM_diff_value_mean                   25
abt_rev_7                             24
BSM_diff_ratio_min                    24
SSS_diff_ratio_mean                   24
DL_Pro_BS_value_min                   23
BSM_diff_ratio_mean                   22
abt_rev_6                             21
BSM_buy_value_min                     20
BSM_diff_ratio_max                    20
BSM_tra_ratio_max                     19
VTR_max                               19
ROI_max                               19
FI_BS_min                             19
DL_Sell_cnt_min                       18
abt_rev_9                             18
SSS_diff_ratio_max                    17
DL_Pro_BS_value_mean                  17
abt_rev_14      

In [45]:
X_train = X_train[var_list]
y_train = train['up_or_down_150']
X_val = X_val[var_list]
y_val = val['up_or_down_150']
X_test = X_test[var_list]
y_test = test['up_or_down_150']

In [118]:
import lightgbm as lgb
gbm = lgb.LGBMClassifier(
        num_leaves=70,
        max_depth =10,
        learning_rate=0.1,
        n_estimators=200,
#         min_child_samples = 15,
        colsample_bytree = 0.8,
#         subsample = 0.8
#         eval_set=[(X_val, y_val)]
        )
gbm.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        importance_type='split', learning_rate=0.1, max_depth=10,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=200, n_jobs=-1, num_leaves=70, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [132]:
from sklearn.metrics import classification_report
X_tra_pred = gbm.predict(X_train)
X_tra_prob = gbm.predict_proba(X_train)
X_tra_pd = pd.DataFrame.from_records(X_tra_prob,columns=['y_0_prob','y_1_prob'])
XY_tra = pd.concat((train[['date','stock']].reset_index(inplace=False,drop=True),pd.DataFrame(X_tra_pred,columns=['y_pred']),X_tra_pd['y_1_prob']),axis=1)
print(classification_report(y_train,X_tra_pred))
X_val_pred = gbm.predict(X_val)
X_val_prob = gbm.predict_proba(X_val)
X_val_pd = pd.DataFrame.from_records(X_val_prob,columns=['y_0_prob','y_1_prob'])
XY_val = pd.concat((val[['date','stock']].reset_index(inplace=False,drop=True),pd.DataFrame(X_val_pred,columns=['y_pred']),X_val_pd['y_1_prob']),axis=1)
print(classification_report(y_val,X_val_pred))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      4109
        1.0       1.00      1.00      1.00      3535

avg / total       1.00      1.00      1.00      7644

             precision    recall  f1-score   support

        0.0       0.54      0.51      0.52       965
        1.0       0.53      0.56      0.54       946

avg / total       0.53      0.53      0.53      1911



/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [185]:
X_test_pred = gbm.predict(X_test)
X_test_prob = gbm.predict_proba(X_test)
X_test_pd = pd.DataFrame.from_records(X_test_prob,columns=['y_0_prob','y_1_prob'])
XY_test = pd.concat((test[['date','stock']].reset_index(inplace=False,drop=True),pd.DataFrame(X_test_pred,columns=['y_pred']),X_test_pd['y_1_prob'],y_test.reset_index(inplace=False,drop=True)),axis=1)
print(classification_report(y_test,X_test_pred))

             precision    recall  f1-score   support

        0.0       0.50      0.65      0.56       906
        1.0       0.56      0.41      0.48      1005

avg / total       0.53      0.52      0.52      1911



/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [186]:
XY_test_order = XY_test.sort_values(['date','y_1_prob'],ascending=[True,False])

In [188]:
XY_test_order.head()

,date,stock,y_pred,y_1_prob,up_or_down_150
208,2018-07-06,1589,1.0,0.960866,0.0
1690,2018-07-06,6414,1.0,0.948526,0.0
143,2018-07-06,1440,1.0,0.883815,1.0
1833,2018-07-06,9914,1.0,0.864433,0.0
793,2018-07-06,2385,1.0,0.818294,1.0


In [194]:
XY_report = XY_test_order[XY_test_order['date'] == '2018-07-06'].iloc[0:10,2:5]

In [195]:
other_date = XY_test_order[~XY_test_order['date'].isin(['2018-07-06','2018-09-28','2018-09-07']) ].date.unique()

In [196]:
other_date

array(['2018-07-13T00:00:00.000000000', '2018-07-20T00:00:00.000000000',
       '2018-07-27T00:00:00.000000000', '2018-08-03T00:00:00.000000000',
       '2018-08-10T00:00:00.000000000', '2018-08-17T00:00:00.000000000',
       '2018-08-24T00:00:00.000000000', '2018-08-31T00:00:00.000000000',
       '2018-09-14T00:00:00.000000000', '2018-09-21T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [197]:
for i in other_date:
    XY_report = pd.concat((XY_report,XY_test_order[XY_test_order['date'] == i].iloc[0:10,2:5]),axis=0)

In [198]:
XY_report.y_pred.value_counts()

1.0    110
Name: y_pred, dtype: int64

In [199]:
print(classification_report(XY_report['up_or_down_150'],XY_report['y_pred'],digits=4))

             precision    recall  f1-score   support

        0.0     0.0000    0.0000    0.0000        48
        1.0     0.5636    1.0000    0.7209        62

avg / total     0.3177    0.5636    0.4063       110



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [121]:
X_test_order = X_test_pd.sort_values('y_1_prob',ascending=False)
y_test_new = y_test.reset_index(inplace=False,drop=True)

In [122]:
for i in [10,20,30,40,50]:
    y_report = y_test_new[y_test_new.index.isin(X_test_order.iloc[0:i,1:2].index)]
    ones = pd.DataFrame(np.ones(i),columns=['y_pred'])
    y_report = pd.concat((y_report.reset_index(inplace=False,drop=True),ones),axis=1)
    print('precision@'+str(i)+'\n')
    print(classification_report(y_report['up_or_down_150'],y_report['y_pred'],digits=4))
    print('\n')

precision@10

             precision    recall  f1-score   support

        0.0     0.0000    0.0000    0.0000         6
        1.0     0.4000    1.0000    0.5714         4

avg / total     0.1600    0.4000    0.2286        10



precision@20

             precision    recall  f1-score   support

        0.0     0.0000    0.0000    0.0000        10
        1.0     0.5000    1.0000    0.6667        10

avg / total     0.2500    0.5000    0.3333        20



precision@30

             precision    recall  f1-score   support

        0.0     0.0000    0.0000    0.0000        14
        1.0     0.5333    1.0000    0.6957        16

avg / total     0.2844    0.5333    0.3710        30



precision@40

             precision    recall  f1-score   support

        0.0     0.0000    0.0000    0.0000        16
        1.0     0.6000    1.0000    0.7500        24

avg / total     0.3600    0.6000    0.4500        40



precision@50

             precision    recall  f1-score   support

        

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [234]:
XY_tra_1 = XY_tra[XY_tra['y_pred'] == 1]

In [235]:
XY_tra_1 = pd.merge(XY_tra_1.drop(['y_pred'],axis=1),pop,on=['stock','date'],how='inner')

In [236]:
XY_tra_1['key'] = XY_tra_1['date'].astype('str') + '||' + XY_tra_1['stock']

In [237]:
from itertools import combinations
stocklist_tra = XY_tra_1.key.unique()
XY_tra_1_permu = list(combinations(stocklist_tra,2))

In [238]:
XY_tra_1_permu_pd = pd.DataFrame.from_records(XY_tra_1_permu,columns=['key_1','key_2'])

In [239]:
XY_tra_1_permu_pd = XY_tra_1_permu_pd[XY_tra_1_permu_pd.key_1.str[12:17] != XY_tra_1_permu_pd.key_2.str[12:17]]

In [240]:
XY_tra_1_permu_100 = XY_tra_1_permu_pd.sample(n=1000000)

In [241]:
stock_market_status = pd.read_pickle('stock_market_status_new.pk')
stock_market_status['key'] = stock_market_status['date'].astype('str') + '||' + stock_market_status['stock']
status = stock_market_status[['key','stock_status']]

In [242]:
status.head()

,key,stock_status
0,2015-10-08||1101,6.62
1,2015-10-08||1102,3.07
2,2015-10-08||1216,0.70
3,2015-10-08||1227,0.27
4,2015-10-08||1262,6.01


In [243]:
XY_tra_final = pd.merge(XY_tra_1_permu_100,XY_tra_1,left_on = 'key_1',right_on = 'key' ,how = 'left')
XY_tra_final = XY_tra_final.drop(['key'],ajxis=1)
XY_tra_final = pd.merge(XY_tra_final,status,left_on = 'key_1',right_on = 'key' ,how = 'left')
XY_tra_final = XY_tra_final.drop(['key'],axis=1)

In [244]:
XY_tra_final = pd.merge(XY_tra_final,XY_tra_1,left_on = 'key_2',right_on = 'key' ,how = 'left',suffixes = ['_1','_2'])
XY_tra_final = XY_tra_final.drop(['key'],axis=1)
XY_tra_final = pd.merge(XY_tra_final,status,left_on = 'key_2',right_on = 'key' ,how = 'left',suffixes = ['_1','_2'])
XY_tra_final = XY_tra_final.drop(['key'],axis=1)

In [245]:
XY_tra_var = pd.DataFrame(XY_tra_final.columns,columns=['var'])

In [246]:
XY_tra_var.to_csv('XY_tra_var.csv')

In [277]:
list(range(len(x_permu)))
stocklist=all_data_select.stock.unique()[0:3]
stock_date=all_data_select.date.unique()[0:3]
x_permu=list(combinations(stocklist,2))
x_len = len(x_permu)
for i in range(x_len):
    s_0 = all_data_select['stock'] == x_permu[i][0]
    s_1 = all_data_select['stock'] == x_permu[i][1]
    stock_1 = all_data_select[s_0].reset_index()
    stock_2 = all_data_select[s_1].reset_index()
    stock_comb = pd.merge(stock_1,stock_2,how = 'left',on = 'date',suffixes = ['_1','_2'])
    exec('stock_comb_{} = stock_comb'.format(i))


,date,stock,y_pred
0,2016-01-08,1101,1.0
1,2016-01-15,1101,1.0
3,2016-01-30,1101,1.0
5,2016-02-19,1101,1.0
6,2016-02-26,1101,1.0


In [280]:
stock_market_status = pd.read_pickle('stock_market_status_new.pk')

In [281]:
stock_market_status.head()

,stock,date,stock_status,market_status,up_or_down,stock_mean_150,up_or_down_150
0,1101,2015-10-08,6.62,1.7,1,1.114178,1
1,1102,2015-10-08,3.07,1.7,1,1.114178,1
2,1216,2015-10-08,0.70,1.7,0,1.114178,0
3,1227,2015-10-08,0.27,1.7,0,1.114178,0
4,1262,2015-10-08,6.01,1.7,1,1.114178,1
